In [21]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
dataset_path = '/content/drive/MyDrive/cv_lab/data'

In [37]:
# Required imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.utils as vutils

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Hyperparameters
nz = 100        # Size of z latent vector (input to generator)
ngf = 64        # Generator feature map size
ndf = 64        # Discriminator feature map size
nc = 3          # Number of channels in the images (3 for RGB)

# Create the generator and discriminator
netG = Generator(nz, ngf, nc).to(device)
netD = Discriminator(nc, ndf).to(device)

# Initialize weights
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)

netG.apply(weights_init)
netD.apply(weights_init)

# Loss function and optimizers
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.00005, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.00005, betas=(0.5, 0.999))

# Load and preprocess your face dataset
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageFolder(root='/content/drive/MyDrive/cv_lab/data', transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2)

# Training loop
num_epochs = 100
fixed_noise = torch.randn(64, nz, 1, 1, device=device)  # Fixed noise for generator performance tracking

for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        # Train discriminator with real images
        netD.zero_grad()
        real_images = real_images.to(device)
        b_size = real_images.size(0)
        labels_real = torch.full((b_size,), 1.0, device=device)
        output_real = netD(real_images).view(-1)
        lossD_real = criterion(output_real, labels_real)
        lossD_real.backward()

        # Train discriminator with fake images
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_images = netG(noise)
        labels_fake = torch.full((b_size,), 0.0, device=device)
        output_fake = netD(fake_images.detach()).view(-1)
        lossD_fake = criterion(output_fake, labels_fake)
        lossD_fake.backward()

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(netD.parameters(), max_norm=1.0)
        optimizerD.step()

        # Train generator
        netG.zero_grad()
        # Feature matching loss
        real_features = netD(real_images).detach()
        fake_features = netD(fake_images)
        lossG = torch.nn.functional.mse_loss(fake_features, real_features)
        lossG.backward()

        # Gradient clipping for generator
        torch.nn.utils.clip_grad_norm_(netG.parameters(), max_norm=1.0)
        optimizerG.step()

    # Print losses for monitoring
    print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{i+1}/{len(dataloader)}] Loss D: {lossD_real.item():.4f}, Loss G: {lossG.item():.4f}")

    # Save sample generated images after each epoch
    if epoch % 10 == 0:
        with torch.no_grad():
            fake_images = netG(fixed_noise).detach().cpu()
            vutils.save_image(fake_images, f"/content/drive/MyDrive/cv_lab/output/fake_images_epoch_{epoch}.png", normalize=True)

print("Training complete!")


Epoch [1/100] Batch [1/1] Loss D: 1.0782, Loss G: 0.3757
Epoch [2/100] Batch [1/1] Loss D: 0.0498, Loss G: 0.5113
Epoch [3/100] Batch [1/1] Loss D: 0.0193, Loss G: 0.6553
Epoch [4/100] Batch [1/1] Loss D: 0.0173, Loss G: 0.8244
Epoch [5/100] Batch [1/1] Loss D: 0.0196, Loss G: 0.8714
Epoch [6/100] Batch [1/1] Loss D: 0.0220, Loss G: 0.9412
Epoch [7/100] Batch [1/1] Loss D: 0.0143, Loss G: 0.9658
Epoch [8/100] Batch [1/1] Loss D: 0.0074, Loss G: 0.9861
Epoch [9/100] Batch [1/1] Loss D: 0.0028, Loss G: 0.9932
Epoch [10/100] Batch [1/1] Loss D: 0.0013, Loss G: 0.9956
Epoch [11/100] Batch [1/1] Loss D: 0.0007, Loss G: 0.9968
Epoch [12/100] Batch [1/1] Loss D: 0.0005, Loss G: 0.9969
Epoch [13/100] Batch [1/1] Loss D: 0.0003, Loss G: 0.9975
Epoch [14/100] Batch [1/1] Loss D: 0.0003, Loss G: 0.9973
Epoch [15/100] Batch [1/1] Loss D: 0.0002, Loss G: 0.9970
Epoch [16/100] Batch [1/1] Loss D: 0.0002, Loss G: 0.9980
Epoch [17/100] Batch [1/1] Loss D: 0.0002, Loss G: 0.9978
Epoch [18/100] Batch [1